# 1 LogistR

In [8]:
import numpy as np
from sklearn.datasets import load_iris
data = load_iris()["data"]
label = load_iris()["target"]

In [9]:
td = data[label != 2]
tl = label[label != 2]
td[1]

array([4.9, 3. , 1.4, 0.2])

In [3]:
def sigmoid(w, x):
    sig = 1 / (np.exp(-np.matmul(x, w)) + 1)
    return sig

In [4]:
# 单样本sgd
def updata_w(data, label, beta=0.01, maxiter=100):
    m, n = data.shape
    w = np.random.randn(n, 1)
    count = 0
    while count < maxiter:
        count += 1
        for indx in range(m):
            i = np.random.randint(0, m)
            pi = sigmoid(w, data[i])
            error = label[i] - pi
            w = w + beta * (1 / m) * np.multiply(np.array([data[i]]).T, error)
    return w
# updata_w(td, tl)

In [5]:
# 全样本 多次循环 进行计算，

In [6]:
def get_loss(m, p, label):
    sums = 0
    for i in range(m):
        # 注意这里的限制，需要注意
        if p[i][0] > 0 or p[i][0] < 1:
            sums += label[i][0] * np.log(p[i][0]) + (1 - label[i][0]) * np.log(1 - p[i][0])
        else:
            sums += 0
    return - 1/m * sums

In [7]:
def updata_w_gd(data, label, beta=0.01, maxiter=100):
    m, n = data.shape
    w = np.random.randn(n, 1)
    count = 0
    label = label.reshape(m, 1)
    
    while count < maxiter:
        count += 1
        p = sigmoid(w, data)
        error = label - p
        w = w + beta * np.matmul(data.T, error) 
        # python3 2个np.ndarray matrix 必须进行matmul
        if count % 5==0:
            loss = get_loss(m, p, label)
            # print("iter is {}, loss is {}".format(count, loss))
        
    return w

ws = updata_w_gd(td, tl)

def predict(test, w, label):
    m, n = test.shape
    rest = sigmoid(w, test)
    # 这里注意不要搞错了
    pred = [1 if i[0] > 0.5 else 0 for i in rest]
    acc = np.sum(pred == label) / m
    return acc, pred, rest
    
acc, pred , rest= predict(td, ws, tl) 
acc 

1.0

# 当 label 为 -1 +1 时的 LR （算法 似乎不太稳定）

In [28]:
ntl = [-1 if i==0 else 1 for i in tl]
ntd = td
ntd[1]

array([4.9, 3. , 1.4, 0.2])

In [245]:
"""
# 换一种方式
def sigmoid(x, y, w):
    temp = np.matmul(np.matmul(x, w).T, y)
    return 1 / (np.exp(-temp))
"""
def sigmoid(x, y, w):
    if isinstance(y, list):
        y = np.array(y).reshape(-1, 1)
    temp = np.multiply(y, np.matmul(x, w))
    return 1 / (1 + np.exp(-temp))


def get_loss_x(m, p, data):
    sums = np.log(p)
    return - 1/m * sums

def get_loss(m, p, label):
    sums = 0
    for i in range(m):
        # 注意这里的限制，需要注意
        if p[i][0] > 0 or p[i][0] < 1:
            sums += np.log(p[i][0])
        else:
            sums += 0
    return -1/m * sums

def updata_w(data, label, beta=0.001, maxiter=100):
    m, n = data.shape
    w = np.random.randn(n, 1)
    count = 0
    label = np.array(label).reshape(m, 1)
    
    while count < maxiter:
        count += 1
        p = sigmoid(data, label, w)
        w = w +  (1 / m) * beta * np.matmul(data.T, np.multiply(label, (1-p))) 
        # python3 2个np.ndarray matrix 必须进行matmul
        if count % 20==0:
            loss = get_loss(m, p, label)
            print("iter is {}, loss is {}".format(count, loss))
    return w
ws  = updata_w(ntd, ntl)

def predict(test, w, label):
    m, n = test.shape
    label = np.array(label).reshape(m, 1)
    rest = sigmoid(test, label, w)
    # 这里注意不要搞错了
    pred = [1 if i[0] > 0.5 else -1 for i in rest]
    acc = np.sum(np.array(pred) == np.array(ntl)) / m
    return acc, pred, rest
    
acc, pred , rest= predict(ntd, ws, ntl) 
acc

iter is 20, loss is 4.662775860212292
iter is 40, loss is 4.468116345624002
iter is 60, loss is 4.273489067589402
iter is 80, loss is 4.078909424230317
iter is 100, loss is 3.8844002707237775


1.0

In [283]:
def sigmoid(x, y, w):
    temp = np.multiply(y, np.matmul(x, w))
    return 1 / (1 + np.exp(-temp))

def updata_w_sgd(data, label, beta=0.001, maxiter=50):
    m, n = data.shape
    w = np.random.randn(n, 1)
    count = 0
    while count < maxiter:
        count += 1
        for indx in range(m):
            i = np.random.randint(0, m)
            pi = sigmoid(data[i], label[i] , w)
            w = w - beta * (1 - pi) * np.multiply(np.array([data[i]]).T, label[i])
            # 这个地方时 是 w = w - alpha * gradient ? 理论上是 + ,但是效果会很差？
    return w

def predict(test, w, label):
    """
    预测结果也不是很稳定
    """
    m, n = test.shape
    label = np.array(label).reshape(m, 1)
    print(label.shape)
    rest = sigmoid(test, label, w)
    # 这里注意不要搞错了
    pred = [1 if i[0] > 0.5 else -1 for i in rest]
    acc = np.sum(np.array(pred) == np.array(ntl)) / m
    return acc, pred, rest
    
wss  = updata_w_sgd(ntd, ntl)
acc, pred , rest= predict(ntd, wss, ntl) 
acc 

(100, 1)


1.0

# 2 GBDT+LR test

In [61]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression

In [62]:
n_trees = 10
clf = GradientBoostingClassifier(n_estimators=n_trees, learning_rate=1.0,
                                 max_depth=4, random_state=0)
result = clf.fit(td, tl)

In [63]:
acc = np.sum(result.predict(td) == tl) / len(tl)
# result.score(td, tl) 等价的
tx = result.apply(td)  # 这个地方注意 
txn = tx.reshape(-1, n_trees)
enc = OneHotEncoder()
re = enc.fit_transform(txn).toarray()

1.0

In [69]:
m1 = LogisticRegression(n_jobs=-1, penalty='l2', C=2, solver='lbfgs')
last_re = m1.fit(txn, tl)
last_re.score(txn, tl)

1.0